# Pipeline configuration

In [20]:
confi_dict = {

    'general':{
        'root_dir':r"C:\Users\na-ch\Desktop\estudio\Master_Big_Data\03_TFM\02_Code\dataset_test",
        'dataset_dir': "CASIA_IrisV1"
    },

    '1_tratarDataset':{
        'show_first' : False
    },

    '1.1_dataAugmentation':{
        'gaussianNoise' : False,
        'stdGN': [5, 10, 15, 20],
        'skipFiles': 1, 
        ''' 
        1 means that you are skiping 1 for the noise augmentation, 
        2 skiping 2 and so on. If you don't want to skip files set it as 0
        '''
        'afinTransformation': False
    },

    '2.1_segmentation':{
        'redNeuronal' : "Iris_unet_d5.h5", 
        'numSamplesGenerator' : 3, 
        'verImagenV1' : False
    }

}

# 1: Tratar el dataset CASIA



## 1 tratar_dataset_casia()

In [2]:
def tratar_dataset_casia(dict):
    
    print("Función 1, tratar_dataset_casia()")
    
    ''' 
    Importando librerías
    '''
    try:
        import os
    except:
        !pip install os
        import os
    try:
        import shutil
    except:
        !pip install shutil
        import shutil
    try:
         import imageio
    except:
        !pip install imageio
        import imageio
    try:
        import matplotlib.pyplot as plt
    except:
        !pip install matplotlib
        import matplotlib.pyplot as plt
    ''' 
    Estableciendo la root de la función
    '''
    os.chdir(dict['general']['root_dir'])
    
    def copy_all_samples(path_samples, destination_directory):
        '''
        Copia todas las muestras del dataset de CASIA a un único directorio.
        '''
        if not os.path.exists(path_samples):
            print("->El directorio indicado como input no existe")
        elif not os.path.exists(destination_directory):
            ''' 
            Comprobamos que el directorio del output no existe
            '''
            os.mkdir(destination_directory) # Creamos un nuevo directorio 
            for root, dirs, files in os.walk(path_samples):
                for file in files:
                    path_file = os.path.join(root,file)
                    shutil.copy2(path_file,destination_directory)   
            print("-> Muestras copiadas satisfactoriamente.")
        elif len(os.listdir(destination_directory)) < 1:
            ''' 
            Comprobamos que el directorio del output no está vacío
            '''
            for root, dirs, files in os.walk(path_samples):
                for file in files:
                    path_file = os.path.join(root,file)
                    shutil.copy2(path_file,destination_directory)   
            print("-> Muestras copiadas satisfactoriamente.")
        else:
            print("-> Muestras previamente copiadas.")

    dict['general']['dataset_unif_dir'] = dict['general']['dataset_dir']+"_unif"

    copy_all_samples(dict['general']['dataset_dir'], dict['general']['dataset_unif_dir'])

    if dict['1_tratarDataset']['show_first']:
        ''' 
        Si en la configuración show_first es True, se mostrará el primer elemento del nuevo directorio.
        '''
        path_img = dict['general']['dataset_unif_dir'] + os.sep + os.listdir(dict['general']['dataset_unif_dir'])[0]
        img = imageio.imread(path_img)
        plt.title("Primer elemento de " + dict['general']['dataset_unif_dir'])
        plt.imshow(img)
    
    ''' 
    Pasando el diccionario con la configuración a la siguiente función
    '''
    return dict


## 1.1 data_augmentation()

In [3]:
def data_augmentation(dict):

    print("Función 1.1, data_augmentation()")
    
    ''' 
    Importando librerías
    '''
    try:
        import os
    except:
        !pip install os
        import os
    try:
        import imageio
    except:
        !pip install imageio
        import imageio
    try:
        import skimage.io as io
    except:
        !pip install scikit-image
        import skimage.io as io
    try:
        import matplotlib.pyplot as plt
    except:
        !pip install matplotlib
        import matplotlib.pyplot as plt
    try:
        from tensorflow.keras.layers import GaussianNoise
    except:
        !pip install tensorflow
        from tensorflow.keras.layers import GaussianNoise
    try:
        import numpy as np
    except:
        !pip install numpy
        import numpy as np
    import random

    ''' 
    Estableciendo la root de la función
    '''
    os.chdir(dict['general']['root_dir'])
    
    ''' 
    Comprobamos si el ruido gaussiano ha sido establecido como True en la configuración
    '''

    if dict['1.1_dataAugmentation']['gaussianNoise']:

        ''' 
        Comprobamos si el ruido gaussiano se ha aplicado anteriormente, si es así, el primer elemento del directorio debería de tener el sufijo _augmentation
        '''
        if "_augment" in os.listdir(dict['general']['dataset_unif_dir'])[1]:
            print("->El ruido gaussiano ya se ha ejecutado con anterioridad")
        else:     
            ''' 
            Desde el directorio unificado creamos nuevas imágenes utilizando el ruido gaussiano como data augmentation.
            Añadimos un contador para que el augmentation sea de 1 cada dos imágenes.
            '''
            i = 0
            for filename in os.listdir(dict['general']['dataset_unif_dir']):
                if dict['1.1_dataAugmentation']['skipFiles'] == 0:
                    path = dict['general']['dataset_unif_dir'] + os.sep + filename
                    image = imageio.imread(path)/255
                    std_rdm = random.choice(dict['1.1_dataAugmentation']['stdGN'])
                    gaussean_function = GaussianNoise(std_rdm/100)
                    noisey = gaussean_function(image.astype(np.float32),training=True)
                    name = path[:-4] + '_augment.bmp'
                    plt.imsave(name, noisey, cmap='gray')
                elif i < dict['1.1_dataAugmentation']['skipFiles']:
                    path = dict['general']['dataset_unif_dir'] + os.sep + filename
                    image = imageio.imread(path)/255
                    std_rdm = random.choice(dict['1.1_dataAugmentation']['stdGN'])
                    gaussean_function = GaussianNoise(std_rdm/100)
                    noisey = gaussean_function(image.astype(np.float32),training=True)
                    name = path[:-4] + '_augment.bmp'
                    io.imsave(name, noisey)
                    i += 1
                else:
                    i = 0
            print("->Data augmentation 'ruido gausseano' ejecutado correctamente")
    else:
        print("->Saltando el ruido gausseano en data augmentation")

    ''' 
    Pasando el diccionario con la configuración a la siguiente función
    '''
    return dict

# 2 Red U-Net v2

## 2.1 segmentation()

In [5]:
def segmentation(dict):

    print("Función 2.1, segmentation()")

    ''' 
    Importando librerías
    '''
    try:
        import os
    except:
        !pip install os
        import os
    try:
        import cv2
    except:
        import cv2
    try:
        import shutil
    except:
        !pip install shutil
        import shutil
    try:
         import imageio
    except:
        !pip install imageio
        import imageio
    try:
        import skimage.transform as trans
        from skimage.io import imshow
        import skimage.io as io
    except:
        !pip install scikit-image
        import skimage.transform as trans
        from skimage.io import imshow
        import skimage.io as io
    try:
        import matplotlib.pyplot as plt
    except:
        !pip install matplotlib
        import matplotlib.pyplot as plt
    try:
        from keras.models import load_model
    except:
        !pip install tensorflow 
        from keras.models import load_model
    try:
        import numpy as np
    except:
        !pip install numpy
        import numpy as np
    try:
        from PIL import Image
    except:
        !pip install Pillow
        from PIL import Image
    ''' 
    Estableciendo la root de la función
    '''
    os.chdir(dict['general']['root_dir'])

    ''' 
    Creamos un generador con un número n de imágenes para pasarselas al modelo en el predict
    *Recordemos que este modelo ya está entrenado para la segmentación del ojo
    '''
    def testGenerator(directory, target_size = (320,320)):
        '''
        Genera las muestras que se le pasarán al método predecir de la red preentrenada.
        
        Parámetros:
        directory -- directorio dónde se encuentran las muestras a generar
        num_image -- número máximo de imágenes que queremos generar
        target_size -- shape de de la muestra
        flag_multi_class -- (de momento mantener pero no nos es útil)
        '''
        for file in os.listdir(directory):
            path_file = os.path.join(directory,file)
            img = imageio.imread(path_file)
            img = img / 255
            img = trans.resize(img,target_size)
            img = np.reshape(img, img.shape+(1,))
            img = np.reshape(img, (1,) + img.shape)
            yield img
    
    def get_samples_names(directory):
        '''
        Devuelve los nombres de las muestras (los que se encuentran en los directorios hoja)
        
        Parámetros:
        directory -- directorio de cuyos subdirectorios hoja se quiere el nombre 
        '''
        names = []
        for file in os.listdir(directory):
            names.append(str(file))
                
        return names

    def saveResult(save_path, name_path, npyfile):
        '''
        Guarda los imágenes segmentadas resultado de la red preentrenada
        
        Parámetros:
        save_path -- ruta en la que se guardarán las muestras segmentadas
        name_path -- directorio del que cogeremos los nombres de las muestras
        npyfile -- resultado de la red preentrenada
        '''
        dim = (320, 280) # dimensioned de las muestras del dataset
        os.mkdir(save_path)
        names = get_samples_names(name_path)
        for i, item in enumerate(npyfile):
            img = item[:,:,0]
            img = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
            io.imsave(os.path.join(save_path,names[i]),img)    

    dict['general']['dataset_unif_seg_dir'] =  dict['general']['dataset_unif_dir']  + "_seg"
    if not os.path.exists(dict['general']['dataset_unif_seg_dir']):
        ''' 
        Cargamos el modelo de segmentación
        '''
        model = load_model(dict['2.1_segmentation']['redNeuronal'])
        print("->Modelo cargado")
        testGene = testGenerator(dict['general']['dataset_unif_dir'])
        Unet_results = model.predict(testGene, verbose=1) 
        saveResult(dict['general']['dataset_unif_seg_dir'], dict['general']['dataset_unif_dir'], Unet_results)
        print("->Segmentaciones guardadas")
    else:
        print("->Segmentaciones previamente guardadas")

    if dict['2.1_segmentation']['verImagenV1']:
        sample = "001_1_1.bmp"
        path_img = dict['general']['dataset_unif_dir'] + os.sep + sample
        path_back = dict['general']['dataset_unif_seg_dir'] + os.sep + sample
        image = Image.open(path_img)
        background = Image.open(path_back)
        background.paste(image, (0, 0), image)
        plt.imshow(background, cmap='gray');
    
    dict['general']['dataset_unif_segv2_dir'] = dict['general']['dataset_unif_seg_dir'] + "v2"
    dict['general']['dataset_unif_segv2_edg_dir'] = dict['general']['dataset_unif_segv2_dir'] + "_edg"

    segv2_dir = False
    edge_dir = False

    for file in os.listdir(dict['general']['dataset_unif_seg_dir']):
        path_file = os.path.join(dict['general']['dataset_unif_seg_dir'],file)
        img = cv2.imread(path_file)
        blur= cv2.GaussianBlur(img,(17,17),0)
        (thresh, binarized) = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY) # se binariza nuevamente
        edges = cv2.Canny(binarized, 10, 255)

        # creamos un nuevo directorio para guardar las muestras segmentadas con mayor calidad
        if not os.path.exists(dict['general']['dataset_unif_segv2_dir']):
            os.mkdir(dict['general']['dataset_unif_segv2_dir'])
            cv2.imwrite(dict['general']['dataset_unif_segv2_dir'] + os.sep + file, binarized)
            segv2_dir = True
        elif segv2_dir:
            cv2.imwrite(dict['general']['dataset_unif_segv2_dir'] + os.sep + file, binarized)
        else:
            print("->La segmentación v2 ya había sido creada")
            break 

        # creamos un nuevo directorio para guardar las muestras con los bordes detectados
        if not os.path.exists(dict['general']['dataset_unif_segv2_edg_dir']):
            os.mkdir(dict['general']['dataset_unif_segv2_edg_dir'])
            cv2.imwrite(dict['general']['dataset_unif_segv2_edg_dir'] + os.sep + file, edges)# guardamos muestra
            edge_dir = True
        elif edge_dir:
            cv2.imwrite(dict['general']['dataset_unif_segv2_edg_dir'] + os.sep + file, edges)# guardamos muestra
        else:
            print("->La edge ya había sido creada")
            break 
    
    print("->Segmentaciones V2 guardadas")
    print("->Edge guardadas")

    return dict

## 2.2 normalization()

In [4]:
def normalization(dict):

    print("Función 2.2, normalization()")

    ''' 
    Importando librerías
    '''
    try:
        import os
    except:
        !pip install os
        import os
    try:
        import cv2
    except:
        import cv2
    try:
        from skimage.transform import (hough_circle, hough_circle_peaks)
        from skimage.io import imshow
    except:
        !pip install scikit-image
        from skimage.transform import (hough_circle, hough_circle_peaks)
        from skimage.io import imshow
    try:
         import imageio
    except:
        !pip install imageio
        import imageio
    try:
        import numpy as np
    except:
        !pip install numpy
        import numpy as np
    try:
        import matplotlib.pyplot as plt
    except:
        !pip install matplotlib
        import matplotlib.pyplot as plt
    ''' 
    Estableciendo la root de la función
    '''
    os.chdir(dict['general']['root_dir'])

    def draw_circles(img, cx, cy, radii):
        '''
        A partir de los centros y el radio detectados dibuja el iris sobre la imagen que se le
        pasa como parámetro.
        '''
        image = img.copy()
        pupil = cv2.circle(image,(cx[0],cy[0]), radii[0]+3, (255, 0, 0), 2)
        iris = cv2.circle(image,(cx[1],cy[1]), radii[1], (255, 0, 0), 2)
        return image
    
    def get_circles(borde, sample):
        path_sample = "CASIA_IrisV1_unif" + os.sep + sample
        path_edged = "CASIA_IrisV1_unif_segv2_edg" + os.sep + sample
        sample_ = imageio.imread(path_sample)
        gray_img = imageio.imread(path_edged)
        
        hough_radii = np.arange(20, 80) # pupila por defecto
        if borde == "iris":
            hough_radii = np.arange(90, 160) # rango del iris
        
        hough_res = hough_circle(gray_img, hough_radii)
        accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=1) 

        
        return [cx[0], cy[0], radii[0]]

    dict['general']['dataset_unif_segv2_edg_norm_dir'] = dict['general']['dataset_unif_segv2_edg_dir'] + "_norm"
    
    os.mkdir(dict['general']['dataset_unif_segv2_edg_norm_dir'])

    for file in os.listdir(dict['general']['dataset_unif_segv2_dir']):
        
        try:
            path_sample = "CASIA_IrisV1_unif" + os.sep + file
            sample_image = imageio.imread(path_sample)
            boundaries,centers = [],[]
            pupil_coord = get_circles("pupil",file)
            iris_coord = get_circles("iris",file)
            cx, cy, radius = list(zip(pupil_coord, iris_coord))
            draw = draw_circles(sample_image,cx,cy,radius)

            boundaries.append(draw)
            centers.append(pupil_coord)

            target = [img for img in boundaries]
            normalized=[]
            cent=0
            
            for img in target:
                #load pupil centers and radius of inner circles
                center_x = centers[cent][0]
                center_y = centers[cent][1]
                radius_pupil=int(centers[cent][2])

                iris_radius = 53 # width of space between inner and outer boundary

                #define equally spaced interval to iterate over
                nsamples = 360
                samples = np.linspace(0,2*np.pi, nsamples)[:-1]
                polar = np.zeros((iris_radius, nsamples))
                for r in range(iris_radius):
                    for theta in samples:
                        #get x and y for values on inner boundary
                        x = (r+radius_pupil)*np.cos(theta)+center_x
                        y = (r+radius_pupil)*np.sin(theta)+center_y
                        x=int(x)
                        y=int(y)
                        try:
                        #convert coordinates
                            polar[r][int((theta*nsamples)/(2*np.pi))] = img[y][x]
                        except IndexError: #ignores values which lie out of bounds
                            pass
                        continue
                res = cv2.resize(polar,(512,64))
                normalized.append(res)
                cent+=1

            h,w = normalized[0].shape # height, width

            roi = normalized[0][10:h, 0:int(512/2)]
            roi = roi.astype(np.uint8) 
            roi_enhanced = cv2.equalizeHist(roi)
            cv2.imwrite(dict['general']['dataset_unif_segv2_edg_norm_dir'] + os.sep + file, roi_enhanced)
        except:
            print("->Ha ocurrido un error durante la etapa de normalización")

            
    
    return dict

        


In [29]:
def normalization(dict):

    print("Función 2.2, normalization()")

    ''' 
    Importando librerías
    '''
    try:
        import os
    except:
        !pip install os
        import os
    try:
        import cv2
    except:
        import cv2
    try:
        from skimage.transform import (hough_circle, hough_circle_peaks)
        from skimage.io import imshow
    except:
        !pip install scikit-image
        from skimage.transform import (hough_circle, hough_circle_peaks)
        from skimage.io import imshow
    try:
         import imageio
    except:
        !pip install imageio
        import imageio
    try:
        import numpy as np
    except:
        !pip install numpy
        import numpy as np
    try:
        import matplotlib.pyplot as plt
    except:
        !pip install matplotlib
        import matplotlib.pyplot as plt
    try:
        import pandas as pd
    except:
        !pip install pandas
        import pandas as pd
    ''' 
    Estableciendo la root de la función
    '''
    os.chdir(dict['general']['root_dir'])

    def draw_circles(img, cx, cy, radii):
        '''
        A partir de los centros y el radio detectados dibuja el iris sobre la imagen que se le
        pasa como parámetro.
        '''
        image = img.copy()
        pupil = cv2.circle(image,(cx[0],cy[0]), radii[0]+3, (255, 0, 0), 2)
        iris = cv2.circle(image,(cx[1],cy[1]), radii[1], (255, 0, 0), 2)
        return image
    
    def get_circles(borde, sample):
        path_sample = "CASIA_IrisV1_unif" + os.sep + sample
        path_edged = "CASIA_IrisV1_unif_segv2_edg" + os.sep + sample
        sample_ = imageio.imread(path_sample)
        gray_img = imageio.imread(path_edged)
        
        hough_radii = np.arange(20, 80) # pupila por defecto
        if borde == "iris":
            hough_radii = np.arange(90, 160) # rango del iris
        
        hough_res = hough_circle(gray_img, hough_radii)
        accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=1) 

        
        return [cx[0], cy[0], radii[0]]

    dict['general']['dataset_unif_segv2_edg_norm_dir'] = dict['general']['dataset_unif_segv2_edg_dir'] + "_norm"
    
    # os.mkdir(dict['general']['dataset_unif_segv2_edg_norm_dir'])

    for file in os.listdir(dict['general']['dataset_unif_segv2_dir']):
        
        try:
            path_sample = "CASIA_IrisV1_unif" + os.sep + file
            sample_image = imageio.imread(path_sample)
            boundaries,centers = [],[]
            pupil_coord = get_circles("pupil",file)
            iris_coord = get_circles("iris",file)
            cx, cy, radius = list(zip(pupil_coord, iris_coord))
            draw = draw_circles(sample_image,cx,cy,radius)

            boundaries.append(draw)
            data = {'pupil':pupil_coord,
                    'iris':iris_coord}
            centers.append(data)

            print(centers)

            target = [img for img in boundaries]
            normalized=[]
            cent=0
            
            for img in target:
                #load pupil centers and radius of inner circles
                center_x = centers[cent][0]
                center_y = centers[cent][1]
                radius_pupil=int(centers[cent][2])

                iris_radius = 53 # width of space between inner and outer boundary

                #define equally spaced interval to iterate over
                nsamples = 360
                samples = np.linspace(0,2*np.pi, nsamples)[:-1]
                polar = np.zeros((iris_radius, nsamples))
                for r in range(iris_radius):
                    for theta in samples:
                        #get x and y for values on inner boundary
                        x = (r+radius_pupil)*np.cos(theta)+center_x
                        y = (r+radius_pupil)*np.sin(theta)+center_y
                        x=int(x)
                        y=int(y)
                        try:
                        #convert coordinates
                            polar[r][int((theta*nsamples)/(2*np.pi))] = img[y][x]
                        except IndexError: #ignores values which lie out of bounds
                            pass
                        continue
                res = cv2.resize(polar,(512,64))
                normalized.append(res)
                cent+=1

            h,w = normalized[0].shape # height, width

            roi = normalized[0][10:h, 0:int(512/2)]
            roi = roi.astype(np.uint8) 
            roi_enhanced = cv2.equalizeHist(roi)
            # cv2.imwrite(dict['general']['dataset_unif_segv2_edg_norm_dir'] + os.sep + file, roi_enhanced)
        except:
            print("->Ha ocurrido un error durante la etapa de normalización")


    # cogemos nombres de las imágenes .bmp
    image_files = []
    new_dir = 'CASIA_IrisV1_unif_segv2_edg_norm'

    for root, dirs, files in os.walk(new_dir):
        for file in files:
            if file.endswith(".bmp"):
                image_files.append(file)
                
    # cogemos las clases a las que pertenece cada .bmp
    classes = []           
    for name in image_files:
        classes.append(name.split('_', 1)[0])
     
    x_pupil, y_pupil, r_pupil =[], [], []
    x_iris, y_iris, r_iris =[],[],[]

    print(centers)

    for i in centers:
        x_pupil.append(i['pupil'][0])
        y_pupil.append(i['pupil'][1])
        r_pupil.append(i['pupil'][2])
        x_iris.append(i['iris'][0])
        y_iris.append(i['iris'][1])
        r_iris.append(i['iris'][2])
    

    data = {'image': image_files,
       'pupil x_center':x_pupil,
       'pupil y_center':y_pupil,
       'pupil radius': r_pupil,
       'iris x_center': x_iris,
       'iris y_center': y_iris,
       'iris radius':r_iris,
       'class': classes}
    
    ################################# arrays must be the same lenght
    df = pd.DataFrame(data)[['image','pupil x_center','pupil y_center','pupil radius','iris x_center', 'iris y_center','iris radius',"class"]] 

    df.to_csv("iris_data.csv")

    return dict

        

# Feature extraction

## extraction()

In [ ]:
def extraction():
    
    print("Función 3, extraction()")

    ''' 
    Importando librerías
    '''
    

## clasification()

# Definir y ejecutar el pipeline

## Definición del pipeline

In [7]:
try:
    from sklearn.preprocessing import FunctionTransformer
    from sklearn.pipeline import Pipeline
except ImportError as e:
    !pip install sklearn
    from sklearn.preprocessing import FunctionTransformer
    from sklearn.pipeline import Pipeline

In [30]:
_1_tratar_dataset_pip = FunctionTransformer(tratar_dataset_casia)
_1_1_data_augmentation_pip = FunctionTransformer(data_augmentation)
_2_1_segmentation_pip = FunctionTransformer(segmentation)
_2_2_normalization_pip = FunctionTransformer(normalization)

iris_recognition_pipeline = Pipeline([('_1_tratarDataset', _1_tratar_dataset_pip), 
                                    ('_1_1_dataAugmentation', _1_1_data_augmentation_pip), 
                                    ('_2_1_segmentation', _2_1_segmentation_pip), 
                                    ('_2_2_mormalization', _2_2_normalization_pip)])

# iris_recognition_pipeline = Pipeline([('_1_tratarDataset', _1_tratar_dataset_pip), 
#                                     ('_2_1_segmentation', _2_1_segmentation_pip), 
#                                     ('_2_2_mormalization', _2_2_normalization_pip)])

# iris_recognition_pipeline = Pipeline([('_2_2_mormalization', _2_2_normalization_pip)])


## Ejecición pipeline


In [31]:
iris_recognition_pipeline.transform(confi_dict)

Función 1, tratar_dataset_casia()
-> Muestras previamente copiadas.
Función 1.1, data_augmentation()
->Saltando el ruido gausseano en data augmentation
Función 2.1, segmentation()
->Segmentaciones previamente guardadas
->La segmentación v2 ya había sido creada
->Segmentaciones V2 guardadas
->Edge guardadas
Función 2.2, normalization()
[{'pupil': [182, 134, 33], 'iris': [176, 135, 105]}]
->Ha ocurrido un error durante la etapa de normalización
[{'pupil': [173, 138, 35], 'iris': [169, 140, 104]}]
->Ha ocurrido un error durante la etapa de normalización
[{'pupil': [174, 120, 35], 'iris': [168, 121, 107]}]
->Ha ocurrido un error durante la etapa de normalización
[{'pupil': [183, 122, 37], 'iris': [179, 122, 104]}]
->Ha ocurrido un error durante la etapa de normalización
[{'pupil': [177, 145, 35], 'iris': [173, 149, 109]}]
->Ha ocurrido un error durante la etapa de normalización
[{'pupil': [179, 133, 36], 'iris': [177, 136, 108]}]
->Ha ocurrido un error durante la etapa de normalización
[{'

ValueError: arrays must all be same length